# InfoObjects Mapping E6Z - PSH

In [2]:
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

## Load data


In [267]:
fp = "/Users/philipp/000_work/060_SciScry/customers/schwarz_it/georg/iobjs/schwarz_bw_iobjs.xlsx"
# Sheet names in Excel
#oh_sname = "OH_IOBJs" 
x6z_sname = "E6Z_IOBJs"
xsh_sname = "PSH_IOBJs"
#xsh_sname = "PSH_IOBJs"
abdon_sname = "abdon_map"
# psh_is_sname = "PSH_IS"
# sh_out_is_sname = "xSH_OUT_IS"

fp_dg = "/Users/philipp/000_work/060_SciScry/customers/schwarz_it/georg/iobjs/hr_iobjs.xlsx"

In [268]:
#df_oh = pd.read_excel(fp, sheet_name=oh_sname)
df_6z = pd.read_excel(fp, sheet_name=x6z_sname)
df_sh = pd.read_excel(fp, sheet_name=xsh_sname)
df_abdon = pd.read_excel(fp, sheet_name=abdon_sname)
df_daniel = pd.read_excel(fp_dg)
df_daniel.fillna(0, inplace=True)
# df_sh_is = pd.read_excel(fp, sheet_name=psh_is_sname)
# df_sh_out_is = pd.read_excel(fp, sheet_name=sh_out_is_sname)

### IOBJ Tables E6Z

In [269]:
fp_prefix = "/Users/philipp/000_work/060_SciScry/customers/schwarz_it/x6z_tables/"

In [284]:
df_rsdiobj = pd.read_excel(fp_prefix+"rsdiobj.xlsx")
rsdiobj_cols = ["IOBJNM","IOBJTP","FIELDNM","MTINFOAREA","TXTSH", "TXTLG"]
df_rsdiobj = df_rsdiobj[rsdiobj_cols]
no_iobj = df_rsdiobj.shape[0] # number of entries

df_iobj = df_rsdiobj[~(df_rsdiobj["IOBJTP"]=="KYF")].reset_index()
df_kyf = df_rsdiobj[df_rsdiobj["IOBJTP"]=="KYF"].reset_index()

no_cha = df_iobj.shape[0]
no_kyf = df_kyf.shape[0]
print(f"characteristics {no_cha}\nkey figures: {no_kyf} ")

if no_iobj != (no_cha+no_kyf):
    raise Exception("IOBJs not matching")

df_rsdcha = pd.read_excel(fp_prefix+"rsdcha.xlsx")
rsdcha_cols = ["CHANM","CHABASNM"]
no_rsdcha = df_rsdcha.shape[0]
df_rsdcha = df_rsdcha[rsdcha_cols]

df_rsdchabas = pd.read_excel(fp_prefix+"rsdchabas.xlsx")
rsdchabas_cols = ["CHABASNM", "DATATP", "INTLEN", "OUTPUTLEN","CONVEXIT","INFOAREA", "ATTRIBFL", "TXTTABFL", "TXTTIMFL"]
df_rsdchabas = df_rsdchabas[rsdchabas_cols]

# combined characteristic properties
df_chas  = df_rsdcha.join(df_rsdchabas, rsuffix="_bas")
df_chas.drop(['CHABASNM_bas'], axis=1, inplace=True)
print(f"No. chas entries {no_rsdcha}")

if df_chas.shape[0] != no_rsdcha:
    raise Exception("missing entries in chas IOBJ properties")
    
df_chas = df_iobj.set_index("IOBJNM").join(df_chas.set_index("CHANM"), how="left")
df_chas.fillna(0, inplace=True)
print(f"Chas Join shape: {df_chas.shape}")

# if df_chas.shape[0] != no_rsdcha:
#     raise Exception("missing entries in chas IOBJ properties")


# KPIs
df_rsdkyf = pd.read_excel(fp_prefix+"rsdkyf.xlsx")
print(f"kyfs entries: {df_rsdkyf.shape}")

df_kpi = df_rsdkyf.set_index("KYFNM").join(df_kyf.set_index("IOBJNM"))
df_kpi.fillna(0, inplace=True)

characteristics 8919
key figures: 3145 
No. chas entries 8641
Chas Join shape: (8919, 15)
kyfs entries: (3145, 20)


In [285]:
df_chas.head()

,index,IOBJTP,FIELDNM,MTINFOAREA,TXTSH,TXTLG,CHABASNM,DATATP,INTLEN,OUTPUTLEN,CONVEXIT,INFOAREA,ATTRIBFL,TXTTABFL,TXTTIMFL
IOBJNM,,,,,,,,,,,,,,,
/CPMB/RENT,4,CHA,/B28/S_RENT,0BWTCT_PLAN,In BPC eing.Kursent.,In BPC eingebettete Kurseinheit,/CPMB/RENT,CHAR,32.0,32.0,0,0BWTCT_PLAN,0,0.0,0.0
/CPMB/RULE,5,CHA,/B28/S_RULE,0BWTCT_PLAN,Geschäftsregel,In BPC eingebettete Geschäftsregel,/CPMB/RULE,CHAR,50.0,50.0,0,0BWTCT_PLAN,0,0.0,0.0
0AA_ACCDET,6,CHA,AA_ACCDET,0FIAA,Kontenfindung,Kontenfindung,0AA_ACCDET,CHAR,8.0,8.0,ALPHA,0FIAA,0,1.0,0.0
0AA_CURYRAQ,7,CHA,AA_CURYRAQ,0FIAA,Kn: Neuzugangstrans.,Kennzeichen: Neuzugangstransaktionen,0AA_CURYRAQ,CHAR,1.0,1.0,0,0,0,0.0,0.0
0AA_DEACT,8,CHA,AA_DEACT,0FIAA,Deaktivierungsdatum,Deaktivierungsdatum,0DATE,CHAR,1.0,1.0,0,0,0,0.0,0.0


In [286]:
df_kpi.head()

,OBJVERS,KYFTP,NCUMFL,FIXCUKY,FIXUNIT,AGGRGEN,AGGREXC,AGGRCHA,KYFDECIM,KYFPRSNT,...,STOCK_COVER_CALCTYPE,STOCK_COVER_PERIODS,DATATP,TXTDEF,index,IOBJTP,FIELDNM,MTINFOAREA,TXTSH,TXTLG
KYFNM,,,,,,,,,,,,,,,,,,,,,
/CPMB/BJCDT,A,NUM,0.0,0,0,SUM,SUM,0,0.0,0.0,...,0.0,0,DEC,0.0,0,KYF,/B28/S_BJCDT,0BWTCT_PLAN,Haben,Haben
/CPMB/BJDBT,A,NUM,0.0,0,0,SUM,SUM,0,0.0,0.0,...,0.0,0,DEC,0.0,1,KYF,/B28/S_BJDBT,0BWTCT_PLAN,Soll,Soll
/CPMB/BJLCDT,A,NUM,0.0,0,0,SUM,SUM,0,0.0,0.0,...,0.0,0,DEC,0.0,2,KYF,/B28/S_BJLCDT,0BWTCT_PLAN,Haben,Haben
/CPMB/BJLDBT,A,NUM,0.0,0,0,SUM,SUM,0,0.0,0.0,...,0.0,0,DEC,0.0,3,KYF,/B28/S_BJLDBT,0BWTCT_PLAN,Soll,Soll
0AB_NDTAX,A,AMO,0.0,0,0,SUM,SUM,0,2.0,0.0,...,0.0,0,CURR,0.0,22,KYF,AB_NDTAX,0TI,n. abzf. Vst,nicht abzugsfähige Vorsteuer in Belegwährung


### clean Daniel IOBJNM field

In [287]:
for i, r in tqdm(df_daniel.iterrows(), total=df_daniel.shape[0]):
    if r["IOBJNM"][:5] == "/BIC/":
        df_daniel.loc[i,"IOBJNM"] = r["IOBJNM"][5:]

  0%|          | 0/25 [00:00<?, ?it/s]

## set required match level

In [288]:
df_6z.head()

,LANGU,IOBJNM,OBJVERS,TXTSH,TXTLG
0,D,/CPMB/BJCDT,A,Haben,Haben
1,D,/CPMB/BJDBT,A,Soll,Soll
2,D,/CPMB/BJLCDT,A,Haben,Haben
3,D,/CPMB/BJLDBT,A,Soll,Soll
4,D,/CPMB/RENT,A,In BPC eing.Kursent.,In BPC eingebettete Kurseinheit


In [275]:
match_lvl = 0.9

### Match via Ratcliff-Obershelp similarity

In [276]:
fp = "/Users/philipp/000_work/060_SciScry/customers/schwarz_it/daniel_geiger/iobj_daniel.xlsx"
df_sh_matches = get_matches(df_src=df_daniel,
                            df_tgt=df_6z,
                           fp=fp)
df_sh_matches.reset_index(inplace=True, drop=True)
df_sh_matches.head()

  0%|          | 0/25 [00:00<?, ?it/s]

Alterscluster
Betriebszugehörigkeit
FB Element
Fach/Führung
Führungsebene
Geschlecht
Kennzahl ID
Kontierungsbene
Nationalität
Organisationseinheit
Planungsrunde
Planungzsenario
UB Element
Geschäftsjahr / Periode
Buchungsperiode
Planabgabejahr (GJ)
Menge (ohne Einheit)
Periodenwert in Hauswährung
Währungsschlüssel Kreiswährung
Periodenwert in Kreiswährung
Mengeneinheit
Währungsschlüssel Hauswährung
Menge
Geschäftsjahr
Geschäftsjahresvariante


,iobj_src,TXTSH_src,iobj_tgt,TXTSH_tgt,match
0,G22MALTCL,Alterscluster,G22MALTCL,Alterscluster,100.000000
1,G22MBTZGK,Betriebszugehörigkeit,G13MBZUGH,Betriebszugehörigkei,97.560976
2,G22MBTZGK,Betriebszugehörigkeit,G22MBTZFL,Betriebszugehörigkei,97.560976
3,G22MBTZGK,Betriebszugehörigkeit,G22MBTZGK,Betriebszugehörigkei,97.560976
4,G22MBTZGK,Betriebszugehörigkeit,G92MBZUGH,Betriebszugehörigkei,97.560976


In [277]:
def get_matches(df_src: pd.DataFrame,
                df_tgt:pd.DataFrame,
                fp:str)->pd.DataFrame:
    """finds the matching IOBJs"""
    df_matches = pd.DataFrame() # results df
    

    for idx_src, row_src in tqdm(df_src.iterrows(), total=df_src.shape[0]):
        
        src_text = df_src[df_src["IOBJNM"]==row_src["IOBJNM"]]["TXTSH"].squeeze()
        print(src_text)
        l = list() # list of comparisons for current IOBJ

        # handle scenario if matching IOBJ in xSH
        df_direct_match = df_tgt[df_tgt["IOBJNM"]==row_src["IOBJNM"]]

#         if df_direct_match.shape[0]==1:
#             df_tmp = {"iobj_src": row_src["IOBJNM"],
#                      "TXTSH_src": src_text,
#                      "iobj_tgt": df_direct_match["IOBJNM"],
#                      "TXTSH_tgt": df_direct_match["TXTSH"],
#                      "match": 0}
#             continue

        # handle non-direct matches
        for idx_tgt, row_tgt in df_tgt.iterrows():
            if (type(row_tgt["TXTSH"]))==np.nan:
                print(f"comparing \'{src_text}\'\nwith \'{row_tgt['TXTSH']}\'")
                continue
            ratio = SequenceMatcher(None,
                                    a=str(src_text),
                                    b=str(row_tgt["TXTSH"])).ratio()


            if ratio < match_lvl: # ensure match quality
                continue
            d_tmp = {"iobj_src": row_src["IOBJNM"],
                     "TXTSH_src": src_text,
                     "iobj_tgt": row_tgt["IOBJNM"],
                     "TXTSH_tgt": row_tgt["TXTSH"],
                     "match": ratio*100}
            l.append(d_tmp)

        if len(l) == 0: # this needs to happen in the outer loop
            df_tmp = {"iobj_src": row_src["IOBJNM"],
                     "TXTSH_src": src_text,
                     "iobj_tgt": "NO_MATCH_FOUND",
                     "TXTSH_tgt": "NO_MATCH_FOUND",
                     "match": 0}

        df_match_tmp = pd.DataFrame(l)
        df_matches = df_matches.append(df_match_tmp)

    #df_matches = df_matches.reset_index()
    df_matches.to_excel(fp,
                       index=False)
    return df_matches

In [298]:
def determine_suitability(df_matches: pd.DataFrame,
                          df_chas: pd.DataFrame,
                          df_kpi: pd.DataFrame) -> pd.DataFrame:
    """ indicates whether datatypes match"""
    
    df_res = pd.DataFrame()
    
    for i, r in tqdm(df_matches.iterrows(), total=df_matches.shape[0]):
        df_tmp = pd.DataFrame()
        src_iobj = r["iobj_src"]
        tgt_iobj = r['iobj_tgt']
        # check whether chas or KPI
        if (src_iobj in df_chas.index) & (tgt_iobj in df_chas.index):
            chas_src = df_chas[df_chas.index==src_iobj].squeeze()
            chas_tgt = df_chas[df_chas.index==tgt_iobj].squeeze()
            
            df_matches.loc[i, "IOBJTP"] = chas_src["IOBJTP"]
    
            df_matches.loc[i, "INTLEN"] = intlen = chas_src["INTLEN"] == chas_tgt["INTLEN"]
            df_matches.loc[i, "OUTPUTLEN"] = outputlen = chas_src["OUTPUTLEN"] == chas_tgt["OUTPUTLEN"]
            df_matches.loc[i, "CONVEXIT"] = convexit = chas_src["CONVEXIT"] == chas_tgt["CONVEXIT"]
            df_matches.loc[i, "INFOAREA"] = infoarea = chas_src["INFOAREA"] == chas_tgt["INFOAREA"]
            df_matches.loc[i, "ATTRIBFL"] = attribfl = chas_src["ATTRIBFL"] == chas_tgt["ATTRIBFL"]   
            df_matches.loc[i, "TXTTABFL"] = txttabfl = chas_src["TXTTABFL"] == chas_tgt["TXTTABFL"]
            df_matches.loc[i, "TXTTIMFL"] = txttimfl = chas_src["TXTTIMFL"] == chas_tgt["TXTTIMFL"]
            
            bool_l = [intlen, outputlen, convexit, infoarea,\
                    attribfl, txttabfl, txttimfl]
            if all([intlen, outputlen, convexit, infoarea,\
                    attribfl, txttabfl, txttimfl]):
                df_matches.loc[i, "prp_match"] = True
            else:
                df_matches.loc[i, "prp_match"] = False

        elif (src_iobj in df_kpi.index) & \
            (tgt_iobj in df_kpi.index):
            kpi_src = df_kpi[df_kpi.index==src_iobj].squeeze()
            kpi_tgt = df_kpi[df_kpi.index==tgt_iobj].squeeze()
            
            df_matches.loc[i, "IOBJTP"] = kpi_src["IOBJTP"]
    
            df_matches.loc[i, "KYFTP"] = kyftp = kpi_src["KYFTP"] == kpi_tgt["KYFTP"]
            df_matches.loc[i, "NCUMFL"] = ncumfl = kpi_src["NCUMFL"] == kpi_tgt["NCUMFL"]
            df_matches.loc[i, "FIXCUKY"] = fixcuky = kpi_src["FIXCUKY"] == kpi_tgt["FIXCUKY"]
            df_matches.loc[i, "FIXUNIT"] = fixunit = kpi_src["FIXUNIT"] == kpi_tgt["FIXUNIT"]
            df_matches.loc[i, "AGGRGEN"] = aggrgen = kpi_src["AGGRGEN"] == kpi_tgt["AGGRGEN"]
            df_matches.loc[i, "AGGREXC"] = aggrexc = kpi_src["AGGREXC"] == kpi_tgt["AGGREXC"]
            df_matches.loc[i, "AGGRCHA"] = aggrcha = kpi_src["AGGRCHA"] == kpi_tgt["AGGRCHA"]
            df_matches.loc[i, "KYFDECIM"] = kyfdecim = kpi_src["KYFDECIM"] == kpi_tgt["KYFDECIM"]
            df_matches.loc[i, "KYFPRSNT"] = kyfprsnt = kpi_src["KYFPRSNT"] == kpi_tgt["KYFPRSNT"]
            
            df_matches.loc[i, "KYFNAV"] = kyfnav = kpi_src["KYFNAV"] == kpi_tgt["KYFNAV"]
            df_matches.loc[i, "UNINM"] = uninm = kpi_src["UNINM"] == kpi_tgt["UNINM"]
            df_matches.loc[i, "DOMANM"] = domanm = kpi_src["DOMANM"] == kpi_tgt["DOMANM"]
            df_matches.loc[i, "PACKFL"] = packfl = kpi_src["PACKFL"] == kpi_tgt["PACKFL"]
            df_matches.loc[i, "KYFSEMANTIC"] = kyfsemantic = kpi_src["KYFSEMANTIC"] == kpi_tgt["KYFSEMANTIC"]
            df_matches.loc[i, "STOCK_COVER_CALCTYPE"] = stock_cover_calctype = kpi_src["STOCK_COVER_CALCTYPE"] == kpi_tgt["STOCK_COVER_CALCTYPE"]
            df_matches.loc[i, "DATATP"] = datatp = kpi_src["DATATP"] == kpi_tgt["DATATP"]
            df_matches.loc[i, "TXTDEF"] = txtdef = kpi_src["TXTDEF"] == kpi_tgt["TXTDEF"]
            
            bool_l = [kyftp, ncumfl, fixcuky, fixunit,\
                        aggrgen, aggrexc, aggrcha, kyfdecim, kyfprsnt,\
                     kyfnav, uninm, domanm, packfl, kyfsemantic, stock_cover_calctype,\
                     datatp, txtdef]
            print(f"all: {all(bool_l)}")
            if all(bool_l):
                df_matches.loc[i, "prp_match"] = True
            else:
                df_matches.loc[i, "prp_match"] = False
            
        else:
            print(f"different type src: {src_iobj} and {tgt_iobj}")
            df_matches.loc[i, "prp_match"] = "incompatible"
            
        #df_matches.to_csv("~/000_work/060_SciScry/customers/schwarz_it/daniel_geiger/iobj_HR_proposals.csv")
        df_matches.to_excel("~/000_work/060_SciScry/customers/schwarz_it/daniel_geiger/iobj_HR_proposals.xlsx", index=False)
    

In [296]:
type(np.nan)

float

In [297]:
determine_suitability(df_sh_matches, df_chas, df_kpi)

df_sh_matches.head()

  0%|          | 0/84 [00:00<?, ?it/s]

match: G22MALTCL - G22MALTCL: attribfl: True

src attribfl: 0 - tgt attribfl: 0
bool attribfl: True
match: G22MBTZGK - G13MBZUGH: attribfl: False

src attribfl: 0 - tgt attribfl: X
bool attribfl: False
match: G22MBTZGK - G22MBTZFL: attribfl: True

src attribfl: 0 - tgt attribfl: 0
bool attribfl: True
match: G22MBTZGK - G22MBTZGK: attribfl: True

src attribfl: 0 - tgt attribfl: 0
bool attribfl: True
match: G22MBTZGK - G92MBZUGH: attribfl: True

src attribfl: 0 - tgt attribfl: 0
bool attribfl: True
different type src: G22MBTZGK and ZXXKBZUGH
match: G22MBTZGK - ZXXMBTZGT: attribfl: True

src attribfl: 0 - tgt attribfl: 0
bool attribfl: True
match: G22MBTZGK - ZXXMBZUGF: attribfl: True

src attribfl: 0 - tgt attribfl: 0
bool attribfl: True
match: G22MBTZGK - ZXXMBZUGH: attribfl: True

src attribfl: 0 - tgt attribfl: 0
bool attribfl: True
match: G22MFBELE - G12MFBELM: attribfl: True

src attribfl: X - tgt attribfl: X
bool attribfl: True
match: G22MFBELE - G22MFBELE: attribfl: True

src attr

,iobj_src,TXTSH_src,iobj_tgt,TXTSH_tgt,match,IOBJTP,INTLEN,OUTPUTLEN,CONVEXIT,INFOAREA,...,KYFDECIM,KYFPRSNT,KYFNAV,UNINM,DOMANM,PACKFL,KYFSEMANTIC,STOCK_COVER_CALCTYPE,DATATP,TXTDEF
0,G22MALTCL,Alterscluster,G22MALTCL,Alterscluster,100.000000,CHA,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G22MBTZGK,Betriebszugehörigkeit,G13MBZUGH,Betriebszugehörigkei,97.560976,CHA,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G22MBTZGK,Betriebszugehörigkeit,G22MBTZFL,Betriebszugehörigkei,97.560976,CHA,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,G22MBTZGK,Betriebszugehörigkeit,G22MBTZGK,Betriebszugehörigkei,97.560976,CHA,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,G22MBTZGK,Betriebszugehörigkeit,G92MBZUGH,Betriebszugehörigkei,97.560976,CHA,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Read available IOBJs in E6Z

In [ ]:
df_e6z_dsos = pd.read_excel("/Users/philipp/000_work/060_SciScry/customers/schwarz_it/georg/openhub_relema.xlsx",
                          sheet_name="OUT_InfoSource_Fields")
df_e6z_dsos.drop(["No"], axis=1, inplace=True)

### Fix IOBJ namings

In [ ]:
for idx, row in tqdm(df_e6z_dsos.iterrows(), total=df_e6z_dsos.shape[0]):
    #print(row["InfoSource IOBJs"][:4])
    if row["InfoSource IOBJs"][:5] == "/BIC/":
        df_e6z_dsos.loc[idx,"IOBJ Name"] = row["InfoSource IOBJs"][5:]
    elif row["InfoSource IOBJs"][:4] == "GXXM":
        df_e6z_dsos.loc[idx,"IOBJ Name"] = row["InfoSource IOBJs"]
    else:
        df_e6z_dsos.loc[idx,"IOBJ Name"] = "0"+row["InfoSource IOBJs"]
        

In [ ]:
#df_sh_matches.set_index("iobj_tgt").join(df_e6z_dsos.set_index("IOBJ Name"), how="outer")
df_sh_6z_match =df_e6z_dsos.set_index("IOBJ Name").join(df_sh_matches.set_index("iobj_tgt"), how="outer")

## drop where empty

In [ ]:
df_sh_6z_match.to_excel("/Users/philipp/000_work/060_SciScry/customers/schwarz_it/georg/openhub_6z_sh_match.xlsx")

In [ ]:
df_e6z_dsos[df_e6z_dsos["IOBJ Name"]=="0FISCVRNT_E"]


## Find IOBJ manually

In [ ]:
df_6z["TXTSH"] = df_6z["TXTSH"].astype("string")

In [ ]:
df_6z[df_6z["TXTSH"].str.contains("gesellschaft")]

In [ ]:
df_is_out = get_matches(df_src=df_sh_out_is,
                            df_tgt=df_6z,
                           fp=fp)

In [ ]:
df_is_out.head()

In [ ]:
df_is_out.to_excel("/Users/philipp/000_work/060_SciScry/customers/schwarz_it/georg/is_out_sh_6z_match.xlsx")